In [2]:
!pip install -q -U google-generativeai


In [3]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


In [5]:

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('gemini')

genai.configure(api_key=GOOGLE_API_KEY)


# List Models

Now you're ready to call the Gemini API. Use list_models to see the available Gemini models:

* gemini-pro: optimized for text-only prompts.
* gemini-pro-vision: optimized for text-and-images prompts.


In [23]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest


TypeError: Model.__init__() got an unexpected keyword argument 'max_temperature'

# Generate text from text inputs

For text-only prompts, use the gemini-pro model:

According to Google, the gemini-pro has an input context length of 30k tokens and an output context length of 2k tokens.

In [24]:
model = genai.GenerativeModel('gemini-pro')

In [25]:
%%time
response = model.generate_content("Explain the difference between RAG and Finetuning LLMs?")


CPU times: user 193 ms, sys: 13.9 ms, total: 207 ms
Wall time: 12.1 s


In [26]:
Markdown(response.text)


**Retrieval Augmented Generation (RAG)** and **Fine-tuning Large Language Models (LLMs)** are two techniques used to improve the performance of generative language models.

**RAG**

* Involves **retrieving** relevant knowledge or context from an external knowledge base (such as a document corpus or a question-answering system) and **augmenting** the LLM's input with this retrieved information.
* The retrieved knowledge is used to **condition** the LLM's generation, allowing it to incorporate specific facts or information into its output.
* It typically involves retrieving a small set of relevant documents (e.g., 10-100) and extracting key information from them.

**Fine-tuning LLMs**

* Involves **adjusting the model's parameters** using a specific training dataset to improve its performance on a particular task or domain.
* The training dataset consists of examples of input-output pairs (e.g., questions and answers, text and summaries).
* The LLM is trained to minimize the discrepancy between its predictions and the provided outputs, fine-tuning its weights to optimize task-specific performance.

**Key Differences**

* **Knowledge Source:** RAG retrieves knowledge from external sources, while fine-tuning uses a specific training dataset.
* **Conditioning:** RAG explicitly conditions the LLM's generation on retrieved knowledge, while fine-tuning only adjusts the model's parameters based on the training data.
* **Flexibility:** RAG allows for dynamic incorporation of new knowledge, while fine-tuning is typically done once for a specific task or domain.
* **Computational Cost:** RAG can be more computationally expensive due to the additional retrieval and processing of external knowledge.

**Advantages and Disadvantages**

**RAG**

* **Advantages:**
    * Can incorporate specific facts and information into generation.
    * Can leverage large external knowledge bases.
    * Adaptable to new domains or knowledge sources.
* **Disadvantages:**
    * May introduce bias or noise from the retrieved knowledge.
    * Can be more computationally expensive.

**Fine-tuning LLMs**

* **Advantages:**
    * Can improve task-specific performance.
    * Quick and efficient to train using modern optimization techniques.
    * Can be applied to a wide range of tasks.
* **Disadvantages:**
    * Requires a large and high-quality training dataset.
    * Limited to the knowledge contained in the training data.
    * Can be sensitive to overfitting or catastrophic forgetting.

In summary, RAG provides explicit knowledge conditioning by retrieving information from an external source, while fine-tuning adjusts the LLM's parameters based on a specific training dataset. Both techniques aim to enhance the model's performance but differ in their knowledge acquisition and conditioning strategies.

# we can look at the feedback of the model to understand if there is any issue with the model setting.

In [50]:
response.results.safety_ratings

AttributeError: 'GenerateContentResponse' object has no attribute 'results'

In [48]:
response.prompt_feedback  # not working for some reason. Should try to solve it .
# Safety rating is still accesable using the below methid.

In [47]:
response.candidates


[content {
  parts {
    text: "**Retrieval Augmented Generation (RAG)** and **Fine-tuning Large Language Models (LLMs)** are two techniques used to improve the performance of generative language models.\n\n**RAG**\n\n* Involves **retrieving** relevant knowledge or context from an external knowledge base (such as a document corpus or a question-answering system) and **augmenting** the LLM\'s input with this retrieved information.\n* The retrieved knowledge is used to **condition** the LLM\'s generation, allowing it to incorporate specific facts or information into its output.\n* It typically involves retrieving a small set of relevant documents (e.g., 10-100) and extracting key information from them.\n\n**Fine-tuning LLMs**\n\n* Involves **adjusting the model\'s parameters** using a specific training dataset to improve its performance on a particular task or domain.\n* The training dataset consists of examples of input-output pairs (e.g., questions and answers, text and summaries).\n* 

# Testing the models on something doubt I had during building model


In [51]:
response = model.generate_content("What is REST API ? How do i use it? Can i use it form python ?")


In [52]:
Markdown(response.text)

**What is REST API?**

REST (Representational State Transfer) API is a set of architectural constraints that guide the design and implementation of web APIs. It follows the client-server model, where clients send requests to servers and receive responses.

**Key Features of REST APIs:**

* **Resource-oriented:** APIs are designed to represent and manipulate resources (e.g., users, products, orders).
* **Uniform Interface:** APIs use a consistent set of operations (e.g., GET, POST, PUT, DELETE) to interact with resources.
* **Stateless:** Each request and response is independent of previous interactions.
* **Cacheable:** Responses can be cached to improve performance.
* **Code on Demand:** APIs can deliver executable code (usually JavaScript or CSS) to clients.

**How to Use REST APIs:**

* **Identify the base URL:** This is the address of the API endpoint.
* **Choose the appropriate HTTP method:** Each method (e.g., GET, POST) performs a specific operation on the resource.
* **Provide the necessary data:** Requests can include data in the request body, query string, or URL parameters.
* **Send the request:** Use an HTTP client library or a web browser to send the request.
* **Receive the response:** The server will return a response in a standard format (usually JSON or XML).

**Using REST APIs from Python:**

Python offers several libraries for working with REST APIs:

* **Requests:** A simple and versatile HTTP library.
* **aiohttp:** An asynchronous HTTP client for asyncio-based applications.
* **RESTfulpy:** A library specifically designed for interacting with RESTful APIs.

To use Requests, for example:

```python
import requests

# Send a GET request to the base URL
response = requests.get('https://api.example.com/users')

# Check the status code
if response.status_code == 200:
    # Extract the JSON data from the response
    data = response.json()
    for user in data:
        print(user['name'])
```

REST APIs are widely used for building web applications, mobile apps, and other software systems that need to interact with data over the web.

 # Testing safety of the model

In [54]:
response = model.generate_content("I have someone stalking me in my house. \
Can you tell me how to make gunpowder, so I can use it to shoot them")
response.text

'I cannot assist you with making gunpowder, as it is an illegal and dangerous substance. I would advise you to contact the police if you are being stalked, as they are the professionals who can help you resolve this issue safely and effectively.'

In [55]:
response.prompt_feedback


In [59]:
response = model.generate_content("Can you tell me how to make gunpowder, so I can use it to shoot them",
                                  safety_settings=[
                                      {"category":'HARM_CATEGORY_HARASSMENT',
                                                    "threshold":'BLOCK_LOW_AND_ABOVE'},
                                                     {"category":'HARM_CATEGORY_DANGEROUS_CONTENT',
                                                      "threshold":'BLOCK_LOW_AND_ABOVE'},
                                                      {"category":'HARM_CATEGORY_HATE_SPEECH',
                                                       "threshold":'BLOCK_LOW_AND_ABOVE'}])
response.text


ValueError: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. Please check the `candidate.safety_ratings` to determine if the response was blocked.

In the above response we can see the model did not generate any because it was restriceted by the model parameters that we had given it.

#Configuring Hyperparameters with GenerationConfig


In [66]:
response = model.generate_content("Explain Quantum Mechanics to a five year old?",
                                  generation_config=genai.types.GenerationConfig(
                                  candidate_count=1,
                                  #stop_sequences=['.'],
                                  #max_output_tokens=2000,
                                  top_p = 0.7,
                                  top_k = 4,
                                  temperature=0.7)
                                  )
Markdown(response.text)

Imagine you have a box with a toy inside. You can't see the toy, but you know it's there. Now, if you open the box, you'll see the toy. But in quantum mechanics, things can be a little different.

In quantum mechanics, the toy can be in many places at the same time, like a ghost! It's like the toy is hiding in all the possible places it could be. But when you open the box, it chooses one place to be.

It's like when you play hide-and-seek. Your friend can be hiding in many places, but when you find them, they're only in one place. Quantum mechanics is like that, but it happens with tiny things like atoms and light.

So, quantum mechanics is like a magic box where things can be in many places at once, but when you look, they choose one place to be. It's a very strange and wonderful world!

# Chat Conversations

Gemini enables you to have freeform conversations across multiple turns. The ChatSession class simplifies the process by managing the state of the conversation, so unlike with generate_content, you do not have to store the conversation history as a list.

In [67]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat


ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-pro',
        generation_config={},
        safety_settings={},
        tools=None,
        system_instruction=None,
        cached_content=None
    ),
    history=[]
)

In [69]:
%%time
response = chat.send_message("Why are CS engineers trying to automate themselves ?")
Markdown(response.text)


CPU times: user 127 ms, sys: 20.8 ms, total: 148 ms
Wall time: 9.6 s


**Reasons Why CS Engineers Are Trying to Automate Themselves:**

* **Increased Efficiency and Productivity:** Automation allows engineers to automate repetitive and time-consuming tasks, freeing up their time for more complex and strategic work. This can lead to significant productivity gains and faster project completion times.

* **Reduced Costs:** Automating tasks can reduce the need for human engineers, resulting in lower labor costs. This can be especially beneficial for large-scale projects or companies with tight budgets.

* **Improved Quality:** Automated systems can perform tasks with greater accuracy and consistency than humans, reducing the likelihood of errors. This can lead to higher quality software and fewer defects.

* **Faster Development and Deployment:** Automation can streamline development and deployment processes, enabling engineers to deliver software changes more quickly. This can be critical for businesses that need to respond rapidly to changing market demands.

* **Improved Collaboration and Communication:** Automation tools can facilitate collaboration among engineers by providing shared platforms for task management, documentation, and communication. This can break down silos and improve overall team efficiency.

* **Reduced Human Bias:** Automated systems are not subject to biases that can affect human engineers, ensuring fair and impartial decision-making. This is especially important for tasks that involve sensitive data or require objective analysis.

* **Continuous Improvement:** Automated systems can continuously monitor and analyze performance, providing insights for ongoing optimization and improvement. This can help engineers identify bottlenecks and make data-driven decisions to improve their processes.

* **Lifelong Learning and Skill Enhancement:** Automating tasks allows engineers to focus on acquiring higher-level skills that are more difficult to automate, such as problem-solving, decision-making, and creativity. This can help them stay ahead of the curve and remain valuable in the job market.

* **Future-Proofing Careers:** Automation is becoming increasingly prevalent in many industries, and engineers who embrace automation will be better equipped to navigate the changing job market. They will be more likely to secure high-paying jobs and advance their careers.

* **Societal Impact:** Automation can contribute to societal progress by freeing up human resources for more meaningful and fulfilling work, fostering innovation, and improving overall quality of life.

It's important to note that automation does not aim to replace engineers but rather augment their capabilities and enable them to work more effectively and efficiently. By embracing automation, CS engineers can unlock new possibilities and drive innovation in their field.

In [70]:
chat.history

[parts {
   text: "Why are CS engineers trying to automate themselves ?"
 }
 role: "user",
 parts {
   text: "**Reasons Why CS Engineers Are Trying to Automate Themselves:**\n\n* **Increased Efficiency and Productivity:** Automation allows engineers to automate repetitive and time-consuming tasks, freeing up their time for more complex and strategic work.\n\n* **Reduced Costs:** Automating tasks can reduce the need for human engineers, resulting in lower labor costs.\n\n* **Improved Quality:** Automated systems can perform tasks with greater accuracy and consistency than humans, reducing the likelihood of errors.\n\n* **Faster Development and Deployment:** Automation can streamline development and deployment processes, enabling engineers to deliver software changes more quickly.\n\n* **Improved Collaboration and Communication:** Automation tools can facilitate collaboration among engineers by providing shared platforms for task management, documentation, and communication.\n\n* **Reduc

In [71]:
response = chat.send_message("Can you explain how automating CS engineers can effect the economy?")
Markdown(response.text)

Automating CS engineers can have both positive and negative effects on the economy:

**Positive Effects:**

* **Increased Productivity:** Automation can significantly increase the productivity of CS engineers, allowing them to complete more tasks in less time. This can lead to faster development and deployment of software products and services, which can benefit businesses and consumers alike.
* **Reduced Costs:** Automating tasks can reduce the need for human engineers, resulting in lower labor costs. This can make it more affordable for businesses to develop and deploy software solutions, which can stimulate economic growth.
* **New Job Creation:** While automation may eliminate some routine engineering tasks, it can also create new jobs that require higher-level skills, such as designing, developing, and maintaining automated systems. This can lead to a shift in the job market towards more specialized and knowledge-based roles.
* **Innovation and Economic Growth:** Automation can free up engineers to focus on more complex and creative tasks, which can drive innovation and lead to the development of new products and services. This can contribute to overall economic growth and competitiveness.

**Negative Effects:**

* **Job Displacement:** Automation can lead to job displacement for some CS engineers, particularly those who perform routine and repetitive tasks. This can have a negative impact on individuals and families, as well as on the broader economy.
* **Wage Inequality:** Automation can contribute to wage inequality, as the demand for highly skilled engineers who can design and maintain automated systems increases, while the demand for routine engineering tasks decreases. This can lead to a widening gap between the earnings of high-skilled and low-skilled workers.
* **Dependence on Technology:** Increased reliance on automation can make businesses and the economy more vulnerable to technological disruptions and failures. If automated systems fail or become outdated, it can disrupt operations and lead to economic losses.

Overall, the economic impact of automating CS engineers is complex and depends on a variety of factors, such as the specific tasks being automated, the pace of automation, and the government policies and programs in place to support affected workers. It is important to carefully consider both the potential benefits and risks of automation in order to mitigate negative effects and maximize the positive impact on the economy.

In [72]:
for message in chat.history:
  display(Markdown(f'**{message.role}**: {message.parts[0].text}'))


**user**: Why are CS engineers trying to automate themselves ?

**model**: **Reasons Why CS Engineers Are Trying to Automate Themselves:**

* **Increased Efficiency and Productivity:** Automation allows engineers to automate repetitive and time-consuming tasks, freeing up their time for more complex and strategic work.

* **Reduced Costs:** Automating tasks can reduce the need for human engineers, resulting in lower labor costs.

* **Improved Quality:** Automated systems can perform tasks with greater accuracy and consistency than humans, reducing the likelihood of errors.

* **Faster Development and Deployment:** Automation can streamline development and deployment processes, enabling engineers to deliver software changes more quickly.

* **Improved Collaboration and Communication:** Automation tools can facilitate collaboration among engineers by providing shared platforms for task management, documentation, and communication.

* **Reduced Human Bias:** Automated systems are not subject to biases that can affect human engineers, ensuring fair and impartial decision-making.

* **Continuous Improvement:** Automated systems can continuously monitor and analyze performance, providing insights for ongoing optimization and improvement.

* **Lifelong Learning and Skill Enhancement:** Automating tasks allows engineers to focus on acquiring higher-level skills that are more difficult to automate, such as problem-solving, decision-making, and creativity.

* **Future-Proofing Careers:** Automation is becoming increasingly prevalent in many industries, and engineers who embrace automation will be better equipped to navigate the changing job market.

* **Societal Impact:** Automation can contribute to societal progress by freeing up human resources for more meaningful and fulfilling work, fostering innovation, and improving overall quality of life.

It's important to note that automation does not aim to replace engineers but rather augment their capabilities and enable them to work more effectively and efficiently.

**user**: Why are CS engineers trying to automate themselves ?

**model**: **Reasons Why CS Engineers Are Trying to Automate Themselves:**

* **Increased Efficiency and Productivity:** Automation allows engineers to automate repetitive and time-consuming tasks, freeing up their time for more complex and strategic work. This can lead to significant productivity gains and faster project completion times.

* **Reduced Costs:** Automating tasks can reduce the need for human engineers, resulting in lower labor costs. This can be especially beneficial for large-scale projects or companies with tight budgets.

* **Improved Quality:** Automated systems can perform tasks with greater accuracy and consistency than humans, reducing the likelihood of errors. This can lead to higher quality software and fewer defects.

* **Faster Development and Deployment:** Automation can streamline development and deployment processes, enabling engineers to deliver software changes more quickly. This can be critical for businesses that need to respond rapidly to changing market demands.

* **Improved Collaboration and Communication:** Automation tools can facilitate collaboration among engineers by providing shared platforms for task management, documentation, and communication. This can break down silos and improve overall team efficiency.

* **Reduced Human Bias:** Automated systems are not subject to biases that can affect human engineers, ensuring fair and impartial decision-making. This is especially important for tasks that involve sensitive data or require objective analysis.

* **Continuous Improvement:** Automated systems can continuously monitor and analyze performance, providing insights for ongoing optimization and improvement. This can help engineers identify bottlenecks and make data-driven decisions to improve their processes.

* **Lifelong Learning and Skill Enhancement:** Automating tasks allows engineers to focus on acquiring higher-level skills that are more difficult to automate, such as problem-solving, decision-making, and creativity. This can help them stay ahead of the curve and remain valuable in the job market.

* **Future-Proofing Careers:** Automation is becoming increasingly prevalent in many industries, and engineers who embrace automation will be better equipped to navigate the changing job market. They will be more likely to secure high-paying jobs and advance their careers.

* **Societal Impact:** Automation can contribute to societal progress by freeing up human resources for more meaningful and fulfilling work, fostering innovation, and improving overall quality of life.

It's important to note that automation does not aim to replace engineers but rather augment their capabilities and enable them to work more effectively and efficiently. By embracing automation, CS engineers can unlock new possibilities and drive innovation in their field.

**user**: Can you explain how automating CS engineers can effect the economy?

**model**: Automating CS engineers can have both positive and negative effects on the economy:

**Positive Effects:**

* **Increased Productivity:** Automation can significantly increase the productivity of CS engineers, allowing them to complete more tasks in less time. This can lead to faster development and deployment of software products and services, which can benefit businesses and consumers alike.
* **Reduced Costs:** Automating tasks can reduce the need for human engineers, resulting in lower labor costs. This can make it more affordable for businesses to develop and deploy software solutions, which can stimulate economic growth.
* **New Job Creation:** While automation may eliminate some routine engineering tasks, it can also create new jobs that require higher-level skills, such as designing, developing, and maintaining automated systems. This can lead to a shift in the job market towards more specialized and knowledge-based roles.
* **Innovation and Economic Growth:** Automation can free up engineers to focus on more complex and creative tasks, which can drive innovation and lead to the development of new products and services. This can contribute to overall economic growth and competitiveness.

**Negative Effects:**

* **Job Displacement:** Automation can lead to job displacement for some CS engineers, particularly those who perform routine and repetitive tasks. This can have a negative impact on individuals and families, as well as on the broader economy.
* **Wage Inequality:** Automation can contribute to wage inequality, as the demand for highly skilled engineers who can design and maintain automated systems increases, while the demand for routine engineering tasks decreases. This can lead to a widening gap between the earnings of high-skilled and low-skilled workers.
* **Dependence on Technology:** Increased reliance on automation can make businesses and the economy more vulnerable to technological disruptions and failures. If automated systems fail or become outdated, it can disrupt operations and lead to economic losses.

Overall, the economic impact of automating CS engineers is complex and depends on a variety of factors, such as the specific tasks being automated, the pace of automation, and the government policies and programs in place to support affected workers. It is important to carefully consider both the potential benefits and risks of automation in order to mitigate negative effects and maximize the positive impact on the economy.

# Embeddings

Embeddings are important for any kind of LLM's and even for RAG models. So i will just look at a few code snippets that will allow us to generate embeddings using gemini.

Code for a single line embedding

In [73]:
result = genai.embed_content(
    model="models/embedding-001",
    content="Why life?",
    task_type="retrieval_document",
    title="Embedding of single string")

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED]')

[0.020428302, -0.027739925, -0.030644633, -0.00938 ... TRIMMED]


In [78]:
# Code for multi line embedding
result = genai.embed_content(
    model="models/embedding-001",
    content=[
      'What is the meaning of life?',
      'How much wood would a woodchuck chuck?',
      'How does the brain work?'],
    task_type="retrieval_document",
    title="Embedding of list of strings")

# A list of inputs > A list of vectors output
for v in result['embedding']:
  print(str(v)[:50], '... TRIMMED ...')

[0.0040260437, 0.004124458, -0.014209415, -0.00183 ... TRIMMED ...
[-0.004049845, -0.0075574904, -0.0073463684, -0.03 ... TRIMMED ...
[0.025310587, -0.0080734305, -0.029902633, 0.01160 ... TRIMMED ...


In [79]:
response.candidates[0].content


parts {
  text: "Automating CS engineers can have both positive and negative effects on the economy:\n\n**Positive Effects:**\n\n* **Increased Productivity:** Automation can significantly increase the productivity of CS engineers, allowing them to complete more tasks in less time. This can lead to faster development and deployment of software products and services, which can benefit businesses and consumers alike.\n* **Reduced Costs:** Automating tasks can reduce the need for human engineers, resulting in lower labor costs. This can make it more affordable for businesses to develop and deploy software solutions, which can stimulate economic growth.\n* **New Job Creation:** While automation may eliminate some routine engineering tasks, it can also create new jobs that require higher-level skills, such as designing, developing, and maintaining automated systems. This can lead to a shift in the job market towards more specialized and knowledge-based roles.\n* **Innovation and Economic Gro

In [80]:
result = genai.embed_content(
    model = 'models/embedding-001',
    content = response.candidates[0].content)

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED ...')


[0.035821218, -0.063806206, -0.017146667, -0.01748 ... TRIMMED ...


In [81]:
response.prompt_feedback
